In [1]:
import lxml.html
import requests
import re
import numpy as np

In [2]:
links = []
categories = ['russia', 'world', 'science', 'media', 'business', 'economics', 'sport', 'culture', 'travel']
texts = ['' for i in range(0,len(categories))]
def get_link (category, year, month, day):
    link = 'https://lenta.ru/rubrics/'+ category + '/'
    link += str(year) + '/'
    if month < 10:
        link += '0' + str(month) + '/'
    else:
        link += str(month) + '/'
    if day < 10:
        link += '0' + str(day) + '/'
    else:
        link += str(day) + '/'
    return link


for category in categories:
    for year in range(2015,2017):
        for month in range(1,13):
            for day in range(1,32):
                link = get_link(category, year, month, day)
                page = requests.get(link)
                tree = lxml.html.fromstring(page.text)
                p = tree.xpath('//div[contains(@class, "g-layout")]//div[contains(@class, "titles")]//a/@href')
                links_current = list(map(lambda a: ['https://lenta.ru' + a, category], p))
                for link in links_current:
                    links.append((link[0], link[1]))
                    


In [3]:
view = [('link', 'S100'), ('category', 'S20')]
links = np.array(links, dtype=view)
print(links)

[(b'https://lenta.ru/news/2015/01/01/putinsspeech/', b'russia')
 (b'https://lenta.ru/news/2015/01/01/elections/', b'russia')
 (b'https://lenta.ru/news/2015/01/01/zarplata1/', b'russia') ...,
 (b'https://lenta.ru/photo/2016/12/04/airports/', b'travel')
 (b'https://lenta.ru/photo/2016/12/04/airports/', b'travel')
 (b'https://lenta.ru/photo/2016/12/04/airports/', b'travel')]


In [4]:
texts = ['' for i in range(0,len(categories))]
for i in range(0,len(categories)):
    category = categories[i]
    category = category.encode('UTF-8')
    link_category = links[links['category'] == category]
    for link in link_category:
        page = requests.get(link[0])
        tree = lxml.html.fromstring(page.text)
        text = tree.xpath('//div[contains(@class, "b-text clearfix js-topic__text")]/p')
        texts[i] += ' ' + [' '.join(list((map(lambda a: str(a.text), text))))][0]        

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import pickle

In [6]:
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(texts)
print(X_counts.shape)

tfidf_transformer = TfidfTransformer(use_idf=False)
X_tfidf = tfidf_transformer.fit_transform(X_counts)
print(X_tfidf.shape)

(9, 439745)
(9, 439745)


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X_tfidf, categories, test_size=0.25)
print(X_counts)

  (0, 386831)	28
  (0, 203537)	28
  (0, 235939)	28
  (0, 185020)	28
  (0, 124205)	28
  (0, 373324)	28
  (0, 108769)	28
  (0, 188458)	28
  (0, 3859)	56
  (0, 83498)	28
  (0, 291520)	28
  (0, 295180)	28
  (0, 56858)	28
  (0, 318584)	28
  (0, 105179)	28
  (0, 34193)	28
  (0, 356426)	28
  (0, 368898)	28
  (0, 184755)	28
  (0, 374620)	28
  (0, 374618)	28
  (0, 161817)	28
  (0, 197655)	28
  (0, 219542)	28
  (0, 195864)	28
  :	:
  (8, 103542)	1926
  (8, 223836)	2
  (8, 416649)	71
  (8, 151062)	391
  (8, 159511)	70
  (8, 378346)	201
  (8, 103441)	546
  (8, 347673)	233
  (8, 423982)	1160
  (8, 384603)	1560
  (8, 120959)	4041
  (8, 389979)	4
  (8, 380285)	1
  (8, 436659)	12
  (8, 337782)	171
  (8, 285738)	4
  (8, 380880)	1
  (8, 405533)	13
  (8, 178992)	191
  (8, 244275)	13
  (8, 238005)	2
  (8, 318536)	263
  (8, 81260)	261
  (8, 337748)	2407
  (8, 300796)	333


In [8]:
SVM = LinearSVC(class_weight='balanced')
classificator = SVM.fit(X_tfidf, categories)

In [9]:
train = ["Американский рэпер Канье Уэст в понедельник, 21 ноября, был госпитализирован в одну из больниц Лос-Анджелеса (штат Калифорния). Как сообщает портал TMZ, в клинике он должен пройти психиатрическое обследование.\
          Вызов на пульт дежурного поступил в 13:20 (00:20 по московскому времени). Музыкант в это время находился вне дома. Прибывшие медики доставили Уэста в медицинский центр Рональда Рейгана при университете Калифорнии. По информации источников портала, решение о госпитализации рэпера было принято «ради его здоровья и безопасности».\
          В правоохранительных органах добавили, что Уэст отказывался ехать в больницу, полицейским пришлось надеть на него наручники.\
          Уэста госпитализировали спустя несколько часов после того, как он отменил все оставшиеся концерты — 21 выступление — своего тура Saint Pablo. Причину своих действий исполнитель объяснять не стал.\
          19 ноября на концерте Сакраменто рэпер исполнил только три композиции и покинул сцену. Перед этим артист обрушился с критикой на кандидата в президенты США от демократов Хиллари Клинтон, поддерживавших ее музыкантов Бейонсе и Джей-Зи, а также американское радиовещание и канал MTV. Его тирада заняла около 10 минут.\
          Канье Уэст — один из популярнейших американских артистов, обладатель 21 награды «Грэмми». В 2016 году он выпустил новый, восьмой альбом The Life Of Pablo."]
print(train)

['Американский рэпер Канье Уэст в понедельник, 21 ноября, был госпитализирован в одну из больниц Лос-Анджелеса (штат Калифорния). Как сообщает портал TMZ, в клинике он должен пройти психиатрическое обследование.          Вызов на пульт дежурного поступил в 13:20 (00:20 по московскому времени). Музыкант в это время находился вне дома. Прибывшие медики доставили Уэста в медицинский центр Рональда Рейгана при университете Калифорнии. По информации источников портала, решение о госпитализации рэпера было принято «ради его здоровья и безопасности».          В правоохранительных органах добавили, что Уэст отказывался ехать в больницу, полицейским пришлось надеть на него наручники.          Уэста госпитализировали спустя несколько часов после того, как он отменил все оставшиеся концерты — 21 выступление — своего тура Saint Pablo. Причину своих действий исполнитель объяснять не стал.          19 ноября на концерте Сакраменто рэпер исполнил только три композиции и покинул сцену. Перед этим арти

In [10]:
counts = count_vect.transform(train)
t = tfidf_transformer.transform(counts)
classificator.predict(t)


array(['culture'], 
      dtype='<U9')

In [11]:
import pickle

In [12]:
with open('clf1.pickle', 'wb') as f:
    pickle.dump(classificator, f)

with open('count_vect1.pickle', 'wb') as f:
    pickle.dump(count_vect, f)
    
with open('transformer1.pickle', 'wb') as f:
    pickle.dump(tfidf_transformer, f)
    